# Develop notebook that checks header keywords that identify a bad/smeared readout.

Exposures with aborted/smeared readouts typically have elapsed 
expsoure times that are different from desired exposure times. 
Elapsed times are between 6-7 seconds regardless of what the 
desired exposure time is. These are easy to visually identify 
for etalon and LFC, but more challenging for dark/thar and 
other calibrations. 

In [2]:
from modules.Utils.kpf_parse import get_datecode
from kpfpipe.models.level0 import KPF0

In [54]:
def L0_bad_readout_check(L0, data_products=['auto'], debug=False):
    """
    This Quality Control function checks if desired readout time
    matches the expected readout time (within some limit). This 
    mismatch idetifies a 'smeared' readout scenario that we want to junk.
    Bad readout states can also have no value for Greed/Red elapsed time.
    Bad readouts have elapsed time between 6 and 7 seconds.
    This occurs a few times per day.
    
    Args:
         L0 - an L0 object
         data_products - L0 data_products to check (list)
                         possible elements = 'auto', 'all',
                                             'Green', 'Red', 'CaHK', 'ExpMeter',
                                             'Guider', 'Telemetry', 'Pyrheliometer'
                                             (note that 'all' should be used rarely since good data
                                              could be missing some extensions, e.g. CaHK, Pyrheliometer)
         debug - an optional flag.  If True, missing data products are noted.

         Did not catch this case: KP.20240926.17416.90, where no data shows up on qlp, but QC tests pass.
     Returns:
         QC_pass - a boolean signifying that the QC passed for failed
    """

    # Check primary header
    Texp_desired = L0.header['PRIMARY']['EXPTIME'] # desired exptime
    Texp_actual  = L0.header['PRIMARY']['ELAPSED'] # actual exposure time
    # print('Desired exposure time: ', Texp_desired)
    # print('Actual exposure time:  ', Texp_actual)

    if (Texp_desired >= 7) and ((Texp_actual > 6.0) & (Texp_actual <= 6.6)):
        QC_pass = False
    else:
        QC_pass = True

    return QC_pass

In [55]:
###  We checked if the green and red elapsed times needed checked, but they are consistent with 'elapsed'

    # # Check primary header, green elapsed time
    # Texp_desired = L0.header['PRIMARY']['GRELAPS'] # desired exptime
    # Texp_actual  = L0.header['PRIMARY']['GRELAPS'] # actual exposure time
    # print('Desired exposure time: ', Texp_desired)
    # print('Actual exposure time:  ', Texp_actual)

    # if (Texp_desired > 7) & (Texp_actual < 7):
    # # if (Texp_actual < 7):        
    #     QC_pass = False
    # else:
    #     QC_pass = True


    # # Check primary header, green elapsed time
    # Texp_desired = L0.header['PRIMARY']['RDELAPS'] # desired exptime
    # Texp_actual  = L0.header['PRIMARY']['RDELAPS'] # actual exposure time
    # print('Desired exposure time: ', Texp_desired)
    # print('Actual exposure time:  ', Texp_actual)

    # if (Texp_desired > 7) & (Texp_actual < 7):
    # # if (Texp_actual < 7):        
    #     QC_pass = False
    # else:
    #     QC_pass = True


In [56]:
# Use the from_fits method from the KPF DRP (not astropy.fits) to read in L0 files

ObsID = 'KP.20241008.31459.57' # Bad Readout etalon file.
L0_filename = '/data/L0/' + get_datecode(ObsID) + '/' + ObsID + '.fits'
L0a = KPF0.from_fits(L0_filename)
print(L0_filename)

ObsID = 'KP.20241008.25014.55' # good KPF file
L0_filename = '/data/L0/' + get_datecode(ObsID) + '/' + ObsID + '.fits'
L0b = KPF0.from_fits(L0_filename)
print(L0_filename)

Texp_desired_a = L0a.header['PRIMARY']['EXPTIME'] # desired exptime
Texp_actual_a  = L0a.header['PRIMARY']['ELAPSED'] # actual exposure time

print("For bad example")
print('Desired exposure time: ', Texp_desired_a)
print('Actual exposure time:  ', Texp_actual_a)


Texp_desired_b = L0b.header['PRIMARY']['EXPTIME'] # desired exptime
Texp_actual_b  = L0b.header['PRIMARY']['ELAPSED'] # actual exposure time

print("For good example")
print('Desired exposure time: ', Texp_desired_b)
print('Actual exposure time:  ', Texp_actual_b)


/data/L0/20241008/KP.20241008.31459.57.fits
/data/L0/20241008/KP.20241008.25014.55.fits
For bad example
Desired exposure time:  60.0
Actual exposure time:   6.464
For good example
Desired exposure time:  60.0
Actual exposure time:   60.018


In [58]:
passfail_1 = L0_bad_readout_check(L0a, data_products=['auto'], debug=False)
print("Pass/Fail 1 (should fail):",passfail_1)
# print(L0a)

passfail_2 = L0_bad_readout_check(L0b, data_products=['auto'], debug=False)
print("Pass/Fail 2 (Should pass):",passfail_2)
# print(L0b)

Desired exposure time:  60.0
Actual exposure time:   6.464
Pass/Fail 1 (should fail): False
Desired exposure time:  60.0
Actual exposure time:   60.018
Pass/Fail 2 (Should pass): True
